In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,15)
import matplotlib.pylab as pl
import xgboost

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv',index_col='id')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv',index_col = 'id')

# EDA using DABL

In [ ]:
!pip install dabl

In [ ]:
import dabl
dabl.plot(df,target_col='target')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['target'] = le.fit_transform(df['target'])

In [ ]:
X = df.drop(columns=['target'])
y = df['target']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
d_train = xgboost.DMatrix(X_train, label=y_train)
d_test = xgboost.DMatrix(X_test, label=y_test)

# XGBoost and SHAP

In [ ]:
import xgboost
params = {
    "eta": 0.01,
    "subsample": 0.5,
     "eval_metric": "logloss"
}
model = xgboost.train(params, d_train, 5000, evals = [(d_test, "test")], verbose_eval=100, early_stopping_rounds=20)

In [ ]:
xgboost.plot_importance(model)
pl.title("xgboost.plot_importance(model)")
pl.show()

In [ ]:
xgboost.plot_importance(model, importance_type="cover")
pl.title('xgboost.plot_importance(model, importance_type="cover")')
pl.show()

In [ ]:
xgboost.plot_importance(model, importance_type="gain")
pl.title('xgboost.plot_importance(model, importance_type="gain")')
pl.show()

In [ ]:
import shap
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

In [ ]:
shap.initjs()

shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])


In [ ]:
shap.force_plot(explainer.expected_value, shap_values[:1000,:], X.iloc[:1000,:])


In [ ]:
shap.summary_plot(shap_values, X, plot_type="bar")


In [ ]:
shap.summary_plot(shap_values, X)


#  ELI5 - ‘Explain like I am a 5-year old’

In [ ]:
import eli5
eli5.show_weights(model) # clf is the model fitted 


In [ ]:
eli5.explain_prediction(model, X_train.iloc[1], feature_names = list(X.columns)) 


# LightGBM and SHAP

In [ ]:
import lightgbm as lgb
d_train = lgb.Dataset(X_train, label=y_train)
d_test = lgb.Dataset(X_test, label=y_test)

In [ ]:
params = {
    "max_bin": 512,
    "learning_rate": 0.05,
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "binary_logloss",
    "num_leaves": 10,
    "verbose": -1,
    "min_data": 100,
    "boost_from_average": True
}

model = lgb.train(params,d_train, 10000, valid_sets=[d_test], early_stopping_rounds=50, verbose_eval=1000)


In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1][0,:], X_train.iloc[0,:])


In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1][:1000,:], X_train.iloc[:1000,:])


In [ ]:
shap.summary_plot(shap_values, X)


In [ ]:
for name in X_train.columns:
    shap.dependence_plot(name, shap_values[1], X, display_features=X)